### This notebook implements the myopic greedy and non-greedy channel allocation strategies ###

#### In order to compare the LSTM-MADRL performance against baseline strategies

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
from scipy.special import jn
import pdb
import json
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm import tqdm
from itertools import combinations
import heapq
import math
#import cvxpy as cp
from pulp import *

### Myopic greedy approach ###
#### Heuristic to find max_A min_u \gamma_u^T.a_u, where \gamma_u is the sum of all SINRs combined for a user u 

In [2]:
N = 8 
U = 3
gamma_dB = np.random.uniform(-3,20,N*U)
gamma_lin = np.power(10,gamma_dB/10)
#plt.hist(gamma_dB)
gamma_mat = np.reshape(gamma_lin, (N,U))
gamma_mat  = np.array([[12.44186053, 21.88359258,  0.57038739],
       [12.36616688, 42.32025383, 20.47811626],
       [ 1.76466114,  7.2071601 , 29.56821724],
       [95.14429372,  1.36205076,  2.07802469],
       [ 1.74174898, 90.22816118,  8.34190772],
       [ 1.1973464 ,  0.79196273,  3.01218801],
       [69.31449097, 95.83876587,  2.95234603],
       [27.0424422 ,  7.88011763,  2.64891699]])




In [3]:
A = np.zeros((N,U))
#1) Initialization: allocate one resource to each user

# 1a) find \gamma_\max,u = \max_n \gamma_{n,u}

gamma_max_us = np.max(gamma_mat, axis = 0)

# 1b) sort in ascending order

gamma_max_us = np.sort(gamma_max_us)

# 1c) initialize set of available resources in  R

R = np.arange(N)

# 1d) in order from worst to best \gamma_\max,u, allocate the best resource of u in \cal R to u, 
#    remove resource from \cal R – update the allocation vector a_u to reflect this

### find the indices where the gamma_max_u is in gamma_mat 
inds = []
resos_to_delete = [] 
for gamma_max_u in gamma_max_us:
    ind = np.where(gamma_max_u ==  gamma_mat )
    inds.append(ind)
    A[ind] = 1.0
    resos_to_delete.append(ind[0][0])
R = np.delete(R, resos_to_delete)

In [4]:
while len(R) != 0:
    #print(R)
    # Allocate the remaining resources

    # 2) find the worst user with current allocation: \hat u = \arg\min_u \gamma_u^T a_u

    dot_prods = []
    for u in range(U):
        dot_prod = np.dot(gamma_mat[:,u],A[:,u])
        dot_prods.append(dot_prod)

    dot_prods = np.array(dot_prods)
    worst_user = np.argmin(dot_prods)

    # 3) find the best resource in \cal R to \hat u, remove resource from \cal R – 
    #    update the allocation vector a_u to reflect this

    reso_vals = []
    for r in R: 
        reso_val = gamma_mat[r, worst_user]
        reso_vals.append(reso_val)
    reso_vals = np.array(reso_vals)
    best_reso = R[np.argmax(reso_vals)]
    A[best_reso, worst_user] = 1.0
    #print('best_reso, worst_user',best_reso, worst_user)
    
    ind_to_delete = np.where(best_reso == R)[0][0]
    #pdb.set_trace()
    R = np.delete(R, ind_to_delete)
    #print(R)
print('Final Allocation matrix for myopic greedy is:')
print(A)

Final Allocation matrix for myopic greedy is:
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]


Myopic non-greedy

Objective Function: Minimize the total number of allocated channel resources.

[ \text{Minimize} \quad \sum_{n=1}^{N}\sum_{u=1}^{U} a_{n,u} ]

SINR Constraint: For each user ( u ), the sum of ( \gamma_{n,u} \times a_{n,u} ) over all channel resources ( n ) must be greater than the SINR threshold ( gamma_thresh_lin ).

[ \sum_{n=1}^{N} \gamma_{n,u} \cdot a_{n,u} \geq gamma_thresh_lin \quad \text{for all} \ u ]

Channel Allocation Constraint: Each channel resource can be allocated to at most one user.

[ \sum_{u=1}^{U} a_{n,u} \leq 1 \quad \text{for all} \ n ]

Relaxed Binary Constraint: ( a_{n,u} ) can take any value between 0 and 1.

[ 0 \leq a_{n,u} \leq 1 \quad \text{for all} \ n, u ]


In [5]:
N = 8
U = 3 
gamma = np.array([[12.44186053, 21.88359258,  0.57038739],
       [12.36616688, 42.32025383, 20.47811626],
       [ 1.76466114,  7.2071601 , 29.56821724],
       [95.14429372,  1.36205076,  2.07802469],
       [ 1.74174898, 90.22816118,  8.34190772],
       [ 1.1973464 ,  0.79196273,  3.01218801],
       [69.31449097, 95.83876587,  2.95234603],
       [27.0424422 ,  7.88011763,  2.64891699]])

gamma_thresh_db = 15 # db
gamma_thresh_lin = np.power(10, gamma_thresh_db/10)

In [6]:
# Define the problem
prob = LpProblem("Channel_Allocation", LpMinimize)

# Create variables (Relax the binary constraint for the LP formulation)
a = LpVariable.dicts("a", (range(N), range(U)), 0, 1, LpInteger)

# Objective Function
prob += lpSum(a[n][u] for n in range(N) for u in range(U))

# SINR Constraints
for u in range(U):
    prob += lpSum(gamma[n][u] * a[n][u] for n in range(N)) >= gamma_thresh_lin, f"SINR_user_{u}"

# Channel Allocation Constraints
for n in range(N):
    prob += lpSum(a[n][u] for u in range(U)) <= 1, f"Channel_resource_{n}"

# Solve the problem using an MILP solver
prob.solve()

# Print the results
#for v in prob.variables():
    #print(v.name, "=", v.varValue)

1

In [7]:
A = np.zeros((N,U))
for n in range(N):
    for u in range(U):
        # The variable names are based on how you defined them earlier
        var = a[n][u]
        A[n][u] = int(var.varValue)  # Convert the float results to binary integer
print('The allocation matrix is:')
print(A)

The allocation matrix is:
[[0. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
